In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import bz2
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
# Load the new file as a dataframe
file_path = '/content/drive/MyDrive/ada/quotes-2020-repub-dem.json.bz2'
df = pd.read_json(file_path, compression='bz2')

In [ ]:
# Add number of quotations per qid to dataframe (Caution: this column is already aggregated)
df['quotationCounts'] = df.groupby(['qids'])['quoteID'].transform("count")

# Visualize the number of quotations per speaker grouped by Democrats and Republicans
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

subplot = axes[0];
subplot.hist(df[df['party'].str[0] == 'Q29468']['quotationCounts'].values, bins = 50, color="lightsteelblue")
subplot.set_title("Quotations per Speaker (Republican)")
subplot.set_xlabel("Quotation Count")
subplot.set_ylabel("Number of Speakers")
subplot.set_xlim([0,45000])
subplot.set_ylim([0,80000])

subplot = axes[1];
subplot.hist(df[df['party'].str[0] == 'Q29552']['quotationCounts'].values, bins = 50, color="cornflowerblue")
subplot.set_title("Quotations per Speaker (Democrat)")
subplot.set_xlabel("Quotation Count")
subplot.set_ylabel("Number of Speakers")
subplot.set_xlim([0,45000])
subplot.set_ylim([0,80000])

In [ ]:
# Top Speakers of the Republican Party
df[df['party'].str[0] == 'Q29468'].groupby(['qids']).mean().sort_values(by='quotationCounts', ascending=False)

In [ ]:
# Top Speakers of the Democratic Party
df[df['party'].str[0] == 'Q29552'].groupby(['qids']).mean().sort_values(by='quotationCounts', ascending=False)